#### model files

[audio-event-recognition/fsd-sinet/](https://essentia.upf.edu/models/audio-event-recognition/fsd-sinet/)    

!wget -q https://essentia.upf.edu/models/audio-event-recognition/fsd-sinet/fsd-sinet-vgg41-tlpf-1.pb

!wget -q https://essentia.upf.edu/models/audio-event-recognition/fsd-sinet/fsd-sinet-vgg41-tlpf-1.json

<https://mtg.github.io/essentia-labs/news/tensorflow/2023/02/08/fsdsinet-models/>


In [1]:
import essentia
print(essentia.__version__)
print(essentia.__file__)
import essentia.standard as es

# let's have a look at what is in there
#print(dir(essentia.standard))


import json
#from queue import Queue
import numpy as np

import utils.util as util
import moviepy.editor as mp

2023-05-05 21:13:26.956635: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-05 21:13:26.956661: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.1-beta6-dev
/home/jtstudents/anaconda3/envs/zugpu2/lib/python3.8/site-packages/essentia/__init__.py


[   INFO   ] MusicExtractorSVM: no classifier models were configured by default
2023-05-05 21:13:27.805534: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 21:13:27.809442: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-05-05 21:13:29.184581: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:07:00.0 name: Tesla V100-DGXS-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.71GiB deviceMemoryBandwidth: 836.37GiB/s
2023-05-05 21:13:29.185595: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 

In [2]:
''' HELPERS '''
print(dir(essentia.standard))
#print(help(es.AudioLoader))

['AfterMaxToBeforeMaxEnergyRatio', 'AllPass', 'AudioLoader', 'AudioOnsetsMarker', 'AudioWriter', 'AutoCorrelation', 'BFCC', 'BPF', 'BandPass', 'BandReject', 'BarkBands', 'BeatTrackerDegara', 'BeatTrackerMultiFeature', 'Beatogram', 'BeatsLoudness', 'BinaryOperator', 'BinaryOperatorStream', 'BpmHistogram', 'BpmHistogramDescriptors', 'BpmRubato', 'CartesianToPolar', 'CentralMoments', 'Centroid', 'ChordsDescriptors', 'ChordsDetection', 'ChordsDetectionBeats', 'ChromaCrossSimilarity', 'Chromagram', 'Chromaprinter', 'ClickDetector', 'Clipper', 'ConstantQ', 'CoverSongSimilarity', 'Crest', 'CrossCorrelation', 'CrossSimilarityMatrix', 'CubicSpline', 'DCRemoval', 'DCT', 'Danceability', 'Decrease', 'Derivative', 'DerivativeSFX', 'DiscontinuityDetector', 'Dissonance', 'DistributionShape', 'Duration', 'DynamicComplexity', 'ERBBands', 'EasyLoader', 'EffectiveDuration', 'Energy', 'EnergyBand', 'EnergyBandRatio', 'Entropy', 'Envelope', 'EqloudLoader', 'EqualLoudness', 'Extractor', 'FFT', 'FFTC', 'Fade

In [10]:
''' FAST PREDICT '''
def fast_predict():
    model_config = {
        
        'graph_filename' : "/raid/DATASETS/.zuble/vigia/zuwav00/fsd-sinet-essentia/models/fsd-sinet-vgg41-tlpf-1.pb",
        'metadata_file' : "/raid/DATASETS/.zuble/vigia/zuwav00/fsd-sinet-essentia/models/fsd-sinet-vgg41-tlpf-1.json",
        
        'batchSize' : 64,
        'lastPatchMode': 'repeat',
        'patchHopSize' : 50
        
    }


    ''' MODEL & METADATA '''
    model = es.TensorflowPredictFSDSINet(
                graphFilename = model_config['graph_filename'],
                batchSize = model_config["batchSize"],
                lastPatchMode = model_config["lastPatchMode"],
                patchHopSize = model_config["patchHopSize"]
                                        )
    metadata = json.load(open(model_config['metadata_file'], "r"))
    labels = metadata["classes"]

    mp4path = '/raid/DATASETS/anomaly/XD_Violence/testing_copy/Lord.of.War__#00-50-10_00-50-50_label_G-0-0.mp4'
    ts = 4
    te = 6
    
    ## FS converter 
    mp4_fs_aac = util.print_acodec_from_mp4([mp4path],only_sr=True) # get audio stream fs from mp4 
    print("\n\n\nmp4_fs_aac",mp4_fs_aac)
    resampler = es.Resample(inputSampleRate=mp4_fs_aac, outputSampleRate=22050)

    
    audio_es = mp.AudioFileClip(filename=mp4path,fps=mp4_fs_aac)
    audio_es_cut = audio_es.subclip(t_start=ts,t_end=te)
    aud_arr2 = audio_es_cut.to_soundarray()
    aud_arr_mono_single2 = np.mean(aud_arr2, axis=1).astype(np.float32)
    aud_arr_essentia = resampler(aud_arr_mono_single2) 

    ## predict
    p_es = model(aud_arr_essentia)#;print("predictions_shape",np.shape(p_es))
    print(np.shape(p_es))
    
    
    def top_from_average(data):
        av = np.mean(data, axis=0)
        sorting = np.argsort(av)[::-1]
        return sorting[:15], [av[i] for i in sorting] ,av

    top_labels_i, averages_sorted , averages = top_from_average(p_es)
    top_labels = [labels[i] for i in top_labels_i]
    top_labels_with_av = [f"{label} ({av:.3f})" for label, av in zip(top_labels, averages_sorted)]
    print(top_labels_with_av)
    
    def top_from_max(data):
        av = np.max(data, axis=0)
        sorting = np.argsort(av)[::-1]
        return sorting[:15], [av[i] for i in sorting] ,av
    
    top_labels_i, averages_sorted , averages = top_from_max(p_es)
    top_labels = [labels[i] for i in top_labels_i]
    top_labels_with_av = [f"{label} ({av:.3f})" for label, av in zip(top_labels, averages_sorted)]
    print(top_labels_with_av,"\n\n\n")
    
    #p_es_max = np.max(p_es, axis=0)
    #for i,label in enumerate(labels):
    #    print(i,label,p_es_max[i])

fast_predict()

2023-05-05 21:57:08.288990: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:07:00.0 name: Tesla V100-DGXS-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.71GiB deviceMemoryBandwidth: 836.37GiB/s
2023-05-05 21:57:08.417233: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:08:00.0 name: Tesla V100-DGXS-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.72GiB deviceMemoryBandwidth: 836.37GiB/s
2023-05-05 21:57:08.458679: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:0e:00.0 name: Tesla V100-DGXS-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.72GiB deviceMemoryBandwidth: 836.37GiB/s
2023-05-05 21:57:08.464022: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 3 with properties: 
pciBusID: 0000:0f:00.0 n




mp4_fs_aac 48000
(4, 200)
['Motor vehicle (road) (0.286)', 'Boom (0.274)', 'Explosion (0.270)', 'Vehicle (0.228)', 'Water (0.124)', 'Car (0.114)', 'Gunshot and gunfire (0.096)', 'Slam (0.080)', 'Mechanisms (0.072)', 'Engine (0.067)', 'Engine starting (0.030)', 'Domestic sounds and home sounds (0.028)', 'Rain (0.027)', 'Door (0.024)', 'Walk and footsteps (0.024)']
['Boom (0.768)', 'Motor vehicle (road) (0.582)', 'Explosion (0.524)', 'Water (0.467)', 'Vehicle (0.370)', 'Car (0.325)', 'Gunshot and gunfire (0.293)', 'Slam (0.256)', 'Engine (0.145)', 'Mechanisms (0.144)', 'Rain (0.102)', 'Engine starting (0.053)', 'Bus (0.047)', 'Walk and footsteps (0.047)', 'Door (0.046)'] 





2023-05-05 21:57:08.880816: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:07:00.0 name: Tesla V100-DGXS-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.71GiB deviceMemoryBandwidth: 836.37GiB/s
2023-05-05 21:57:08.881773: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:08:00.0 name: Tesla V100-DGXS-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.72GiB deviceMemoryBandwidth: 836.37GiB/s
2023-05-05 21:57:08.884082: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:0e:00.0 name: Tesla V100-DGXS-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.72GiB deviceMemoryBandwidth: 836.37GiB/s
2023-05-05 21:57:08.893138: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 3 with properties: 
pciBusID: 0000:0f:00.0 n